In [3]:
import drawSvg as draw
from drawSvg.widgets import DrawingWidget
import numpy as np

canvas_size=(800,800)
# Create drawing
d = draw.Drawing(*canvas_size, origin='center')

linkage = draw.Group()
d.append(linkage)
traj=draw.Group()
d.append(traj)


def redraw_linkage(nodes=[], edges=[],node_radius=6,edge_width=5):
    linkage.children.clear()
    for id1,id2 in edges:
        linkage.draw(draw.Line(*nodes[id1],*nodes[id2],stroke='gray',stroke_width=edge_width))
    for x,y in nodes[1:-1]:
        linkage.draw(draw.Circle(x,y,node_radius,fill='green'))
    x,y=nodes[0]
    linkage.draw(draw.Circle(x,y,node_radius,fill='blue'))
    x,y=nodes[-1]
    linkage.draw(draw.Circle(x,y,node_radius,fill='orange'))

def redraw_trajectory(drive_node=[],end_node=[]):
    traj.children.clear()
    for p0,p1 in zip(drive_node[:-1],drive_node[1:]):
        traj.draw(draw.Line(*p0,*p1,stroke='blue',stroke_width=3))
    for p0,p1 in zip(end_node[:-1],end_node[1:]):
        traj.draw(draw.Line(*p0,*p1,stroke='orange',stroke_width=3))
        
        
def rotation_matrix(theta=0):
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))
    return R

def simulate_linkage(nodes=[],edges=[],lengths=[]):
    for i in range(0,len(edges),2):
        e02,e12=edges[[i,i+1],:]
        l02,l12=lengths[[i,i+1]]
        p0,p1=nodes[[e02[0],e12[0]],:]
        if np.linalg.norm(p1-p0)>l02+l12:
            p0=p1-(l02+l12)*(p1-p0)/np.linalg.norm(p1-p0)
            nodes[e02[0],:]=p0
        v01=(p1-p0)/np.linalg.norm(p1-p0)
        l01=np.linalg.norm(p1-p0)
        phi=np.arccos(max(-1,min(1,(l01**2+l02**2-l12**2)/(2*l01*l02))))
        p2=rotation_matrix(phi)@v01*l02+p0
        nodes[e02[1],:]=p2

In [4]:
nodes=np.array([(100,100),(0,0),(0,0),(0,0),(0,0),(0,0),(0,0)])
edges=np.array([(0,2),(1,2),(2,3),(1,3),(1,4),(0,4),(3,5),(4,5),(5,6),(4,6)])
lengths=np.array([100,100,100,100,100,150,100,100,150,100])
drive_node=[]
end_node=[]
# Create interactive widget and register mouse events
widget = DrawingWidget(d)
@widget.mousedown
def mousedown(widget, x, y, info):
    drive_node.append([*nodes[0]])
    if len(drive_node)>1:
        for i in range(0,10):
            nodes[0,:]=(1-i/10.0)*np.asarray(drive_node[-2])+(i/10.0)*np.asarray(drive_node[-1])
            simulate_linkage(nodes,edges,lengths)
            end_node.append([*nodes[-1]])
        nodes[0,:]=drive_node[-1]
    else:
        end_node.append([*nodes[-1]])
    redraw_trajectory(drive_node,end_node)
    redraw_linkage(nodes,edges)
    widget.refresh()
@widget.mousemove
def mousemove(widget, x, y, info):
    nodes[0]=(x,y)
    simulate_linkage(nodes,edges,lengths)
    redraw_trajectory(drive_node,end_node)
    redraw_linkage(nodes,edges)
    widget.refresh()
widget

DrawingWidget()